In [5]:
input_dir = '/home/petadimensionlab/Desktop/PRJNA320132'
output_dir =  '/home/petadimensionlab/Desktop/PRJNA320132'
## reference file ##
gg97fasta = '/home/petadimensionlab/ws/ref/gg_13_5_otus/rep_set/97_otus.fasta'
gg97tax = '/home/petadimensionlab/ws/ref/gg_13_5_otus/taxonomy/97_otu_taxonomy.txt'

In [6]:
import os, glob
#### configurations ####
is_taxonomy = 'yes'
is_denovopicking = 'yes'
is_mapping = 'yes'
is_deletefiles = 'yes'
## softwares ##
vsearch = '/home/petadimensionlab/miniconda3/pkgs/vsearch-2.7.0-1/bin/vsearch'
## input / output file ##
config_file_name = 'config.csv'

In [7]:
## parameters ##
maxdiffs = 5 #10
minmergelen = 1
maxmergelen = 600
minseqlength = 50
maxee_rate = 0.5/100 #%
minsize = 2
thr = 0.97
threadsnum = 4
## file names ##
filtered_file_name = 'filtered.fasta'
centroids_file_name = 'centroids.fasta'
denovootu_file_name = 'denovootu.txt'
hits_file_name = 'hits.txt'
ggotuname_file_name = 'ggotu_name.txt'
ggotuID_file_name = 'ggotu.txt'

## filtering ##
filtered_file = os.path.join(output_dir,filtered_file_name)
centroids_file = os.path.join(output_dir,centroids_file_name)

## Assign taxonomy ##
hits_file = os.path.join(output_dir,hits_file_name)
if is_taxonomy=='yes':
    cmd = '%s --usearch_global %s --db %s --strand both --id %s --threads %s --userfields \"query+target+id\" --userout %s' % (vsearch,centroids_file,gg97fasta,str(thr),str(threadsnum),hits_file)
    os.system(cmd)
else:
    msg = 'Skip taxonomy assignment.'
    print( msg )

## de novo picking ##
denovootu_file = os.path.join(output_dir,denovootu_file_name)
if is_denovopicking=='yes':
    cmd = '%s --usearch_global %s --db %s --strand both --id %s --threads %s --otutabout %s' % (vsearch,filtered_file,centroids_file,str(thr),str(threadsnum),denovootu_file)
    os.system(cmd)
else:
    msg = 'Skip denovo picking.'
    print( msg )

## Mapping to GreenGene database ##
if is_mapping=='yes':
    ggid2ggname = {}
    frgg97 = open(gg97tax,'r').readlines()
    for line in frgg97:
        line = line.replace('\n','')
        tmp = line.split('\t')
        ggid2ggname[tmp[0]] = tmp[1]
    ## make a mapping from "user defined sequence ID" to "greengene ID"
    myid2ggid = {}
    myid2ggname = {}
    frh = open(hits_file,'r').readlines()
    for line in frh:
        tmp = line.split('\t') # ggID: tmp[1]
        ggid = tmp[1]
        ggname = ggid2ggname[ggid]
        myids = tmp[0].split(';') # myID : tmp[0]
        myid2ggid[myids[0]] = ggid
        myid2ggname[myids[0]] = ggname

    frd = open(denovootu_file,'r').readlines()
    header = frd[0] # restore header information
    frd.pop(0) # remove the header line
    ## taxonomy name version ##
    ggotu_name_file = os.path.join(output_dir,ggotuname_file_name)
    fw = open(ggotu_name_file,'w')
    fw.write(header)
    for line in frd:
        tmp = line.split('\t') # tmp[0]: myID
        try:
            tmp[0] = myid2ggname[tmp[0]]
            fw.write('\t'.join(tmp))
        except:
            continue
    fw.close()
    ## taxonomy ID version ##
    ggotu_ID_file = os.path.join(output_dir,ggotuID_file_name)
    fw = open(ggotu_ID_file,'w')
    fw.write(header)
    for line in frd:
        tmp = line.split('\t') # tmp[0]: myID
        try:
            tmp[0] = myid2ggid[tmp[0]]
            fw.write('\t'.join(tmp))
        except:
            continue
    fw.close()
else:
    msg = 'Skip taxonomy mapping.'
    print( msg )


In [8]:
## delete intermediate files ##
if is_deletefiles=='yes':
    paired_files = glob.glob('paired*.fastq.gz')
    unpaired_files = glob.glob('unpaired*.fastq.gz')
    merged_files = glob.glob('*_merged.fasta')
    rm_files = ['appended.fastq','filtered.fasta','unique.fasta']+paired_files+unpaired_files+merged_files
    for file in rm_files:
        cmd = 'rm %s' % (file)
        os.system(cmd)